In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

##########################
# 1. 데이터 불러오기
##########################
base_dir = "covid19-radiography-database/COVID-19_Radiography_Dataset"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

##########################
# 2. 모델 정의 (축소 버전)
##########################
model = models.Sequential()
# 첫 번째 합성곱 레이어: 필터 수를 32 -> 16으로 축소
model.add(layers.Conv2D(16, (3,3), activation='relu', input_shape=(224,224,3)))
model.add(layers.MaxPooling2D((2,2)))

# 두 번째 합성곱 레이어: 필터 수를 64 -> 32로 축소
model.add(layers.Conv2D(32, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

# 세 번째 합성곱 레이어: 필터 수를 128 -> 64로 축소
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
# 완전연결 레이어: 유닛 수를 128 -> 64로 축소
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

##########################
# 3. 모델 컴파일
##########################
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

model.summary()

##########################
# 4. 콜백 설정
##########################
checkpoint_path = "best_small_model.h5"
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

##########################
# 5. 모델 학습
##########################
epochs = 50
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stopping]
)

##########################
# 6. (선택) 최종 모델 저장
##########################
model.save("covid_classification_final_small.h5")
print("축소된 최종 모델이 covid_classification_final_small.h5로 저장되었습니다.")


2025-01-26 16:38:50.783466: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-26 16:38:50.783527: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-26 16:38:50.783566: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-26 16:38:50.791505: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-26 16:38:51.798139: W tensorflow/compiler/

Found 33866 images belonging to 4 classes.
Found 8464 images belonging to 4 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        18496     
                                     

2025-01-26 16:38:53.785760: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/50
2117/2117 [==============================] - ETA: 0s - loss: 0.7807 - accuracy: 0.6755
Epoch 1: val_loss improved from inf to 0.59456, saving model to best_small_model.h5
2117/2117 [==============================] - 7276s 3s/step - loss: 0.7807 - accuracy: 0.6755 - val_loss: 0.5946 - val_accuracy: 0.7642


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/50
2117/2117 [==============================] - ETA: 0s - loss: 0.5978 - accuracy: 0.7632
Epoch 2: val_loss improved from 0.59456 to 0.45734, saving model to best_small_model.h5
2117/2117 [==============================] - 7088s 3s/step - loss: 0.5978 - accuracy: 0.7632 - val_loss: 0.4573 - val_accuracy: 0.8306
Epoch 3/50
2117/2117 [==============================] - ETA: 0s - loss: 0.5078 - accuracy: 0.7964
Epoch 3: val_loss did not improve from 0.45734
2117/2117 [==============================] - 7053s 3s/step - loss: 0.5078 - accuracy: 0.7964 - val_loss: 0.4905 - val_accuracy: 0.8217
Epoch 4/50
2117/2117 [==============================] - ETA: 0s - loss: 0.4216 - accuracy: 0.8346
Epoch 4: val_loss improved from 0.45734 to 0.43057, saving model to best_small_model.h5
2117/2117 [==============================] - 7055s 3s/step - loss: 0.4216 - accuracy: 0.8346 - val_loss: 0.4306 - val_accuracy: 0.8461
Epoch 5/50
2117/2117 [==============================] - ETA: 0s - loss: 0.3281

KeyboardInterrupt: 